In [ ]:
!pip install --upgrade pip
!pip install fosforml numpy pandas matplotlib scikit-learn seaborn python-dateutil
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 
!pip install fosforio
!pip install refractio
!pip install refractml

In [ ]:
!pip install seaborn scipy xgboost pandas dice-ml tabulate numpy scikit-learn pandas-profiling plotly matplotlib scipy statsmodels seaborn pydantic-settings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [2]:
from refractio import snowflake
snowflake.get_connection(connection_name="Final hr attrition data")

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
Exception occurred in getting snowflake connection: 'connectionSources'


In [3]:
df = snowflake.get_dataframe("Final_HR_Attrition_Data")
df

,SALARY,SENIORITY,TENURE_MONTHS,MONTHS_AFTER_COLLEGE,BIRTH_YEAR,AGE,MAPPED_ROLE_CLEAN,SEX,ETHNICITY,HOSPITAL_TYPE,...,OVER_TIME_HOURS,CHURN,BUSINESS_TRAVEL,ENVIRONMENT_SATISFACTION,JOB_SATISFACTION,MARTIAL_STATUS,PERCENTAGE_SALARY_HIKE,PERFORMANCE_RATING,RELATIONSHIP_SATISFACTION,WORK_LIFE_BALANCE
0,31657.073,1,13,161,1986,38,occupational,F,Black,Psychiatric,...,7,True,Travel_Rarely,High,Medium,Married,18,Excellent,Medium,Better
1,82697.324,2,10,72,1988,36,social,M,API,Acute Care - Department of Defense,...,6,False,Travel_Rarely,High,Very High,Married,19,Excellent,Low,Good
2,67291.312,1,65,7,1986,38,nurse,M,Black,Acute Care - Department of Defense,...,9,False,Non-Travel,High,Low,Married,14,Excellent,Very High,Bad
3,41186.945,1,36,33,1988,36,technologist,F,Hispanic,Psychiatric,...,8,False,Travel_Rarely,Low,Very High,Divorced,21,Low,Medium,Bad
4,54826.128,2,9,23,1990,34,physician,M,Black,Childrens,...,11,True,Travel_Rarely,High,Very High,Single,16,Good,High,Better
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,97520.177,5,45,113,1969,55,physician,M,Hispanic,Acute Care Hospitals,...,1,False,Travel_Rarely,High,High,Divorced,20,Outstanding,Low,Best
299996,55666.168,2,32,5,1987,37,emt,M,White,Childrens,...,9,True,Travel_Rarely,Medium,Low,Married,23,Low,Very High,Best
299997,49866.716,2,34,4,1986,38,physician,F,White,Childrens,...,0,True,Travel_Rarely,Medium,Medium,Single,14,Excellent,Medium,Bad
299998,77706.113,2,6,26,1996,28,physician,F,API,Childrens,...,1,False,Travel_Rarely,High,High,Single,11,Good,Very High,Good


In [4]:
print(df.isnull().sum())

SALARY                           0
SENIORITY                        0
TENURE_MONTHS                    0
MONTHS_AFTER_COLLEGE             0
BIRTH_YEAR                       0
AGE                              0
MAPPED_ROLE_CLEAN                0
SEX                              0
ETHNICITY                        0
HOSPITAL_TYPE                    0
HOSPITAL_OWNERSHIP               0
COMPANY_NAME                     0
CITY                             0
STATE                            0
DISTANCE                         0
DEGREE_CLEAN                     0
SCHOOL_END_DATE                  0
JOB_START_DATE                   0
JOB_END_DATE                 90102
USER_ID                          0
OVER_TIME_HOURS                  0
CHURN                            0
BUSINESS_TRAVEL                  0
ENVIRONMENT_SATISFACTION         0
JOB_SATISFACTION                 0
MARTIAL_STATUS                   0
PERCENTAGE_SALARY_HIKE           0
PERFORMANCE_RATING               0
RELATIONSHIP_SATISFA

In [5]:
df = df.dropna()

In [6]:
print(df.isnull().sum())

SALARY                       0
SENIORITY                    0
TENURE_MONTHS                0
MONTHS_AFTER_COLLEGE         0
BIRTH_YEAR                   0
AGE                          0
MAPPED_ROLE_CLEAN            0
SEX                          0
ETHNICITY                    0
HOSPITAL_TYPE                0
HOSPITAL_OWNERSHIP           0
COMPANY_NAME                 0
CITY                         0
STATE                        0
DISTANCE                     0
DEGREE_CLEAN                 0
SCHOOL_END_DATE              0
JOB_START_DATE               0
JOB_END_DATE                 0
USER_ID                      0
OVER_TIME_HOURS              0
CHURN                        0
BUSINESS_TRAVEL              0
ENVIRONMENT_SATISFACTION     0
JOB_SATISFACTION             0
MARTIAL_STATUS               0
PERCENTAGE_SALARY_HIKE       0
PERFORMANCE_RATING           0
RELATIONSHIP_SATISFACTION    0
WORK_LIFE_BALANCE            0
dtype: int64


In [7]:
df = df.drop(["USER_ID", "JOB_START_DATE", "JOB_END_DATE", "SCHOOL_END_DATE"], axis = 1)

In [10]:
CATEGORICAL_COLUMNS = [
    "MAPPED_ROLE_CLEAN", "SEX", "ETHNICITY", "HOSPITAL TYPE", "HOSPITAL OWNERSHIP",
    "COMPANY NAME", "CITY", "STATE", "DISTANCE", "DEGREE CLEAN", "BUSINESS TRAVEL",
    "ENVIRONMENT SATISFACTION", "JOB SATISFACTION", "MARTIAL STATUS", "PERFORMANCE_RATING",
    "RELATIONSHIP SATISFACTION", "WORK_LIFE_BALANCE"
]
NUMERICAL_COLUMNS = [
    "SALARY", "SENIORITY", "TENURE MONTHS", "MONTHS_AFTER_COLLEGE",
    "BIRTH_YEAR", "AGE", "OVER TIME_HOURS", "PERCENTAGE_SALARY_HIKE"
]
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]
 
feature_columns = [col for col in CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS if col in df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in df.columns]
 
X = df[feature_columns]
y = df[LABEL_COLUMNS].values.ravel()

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
 
# Define transformers
categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
)
 
numerical_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(clip=True)
)
 
# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, CATEGORICAL_COLUMNS),
        ('num', numerical_transformer, NUMERICAL_COLUMNS)
    ]
)
 
# Create pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])

In [13]:
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [3, 5, 7],
    'classifier__learning_rate': [0.01, 0.1, 0.2]
}
 
from sklearn.model_selection import GridSearchCV
 
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)
 
best_pipeline = grid_search.best_estimator_

ValueError: 
All the 135 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "/tmp/pip_packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'HOSPITAL TYPE'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/tmp/pip_pa